# Pre-processing boston marathon Data-set

The aim of this is to process the three results available of boston marathon here:

    - https://www.kaggle.com/rojour/boston-results
    
With the objective of split our information in this couple of segments:
    - 5k
    - 10k
    - 15k
    - 20k
    - 25k
    - 30k
    - 35k
    - 40k

One each of this DataSet will include this factors:
    - Relative temperature. Using wind speed, humidity, and other climate factors.
    - Humidity. %Humidity, maybe this factor is redundant because it is included to calculate te relative temperature.
    - Gender. Male/Female.
    - Age. Using Years.
    - Elevation of his/her birth city. Using meters.
    
    - Finally time in that he did the corresponding partial.
    

In [ ]:
import pandas as pd
import numpy as np

- First of all, add a label of which year is this results in the aim to diference the diferent runners.

In [10]:
results_2015 = pd.read_csv("data_sets/marathon_results_2015.csv")
results_2016 = pd.read_csv("data_sets/marathon_results_2016.csv")
results_2017 = pd.read_csv("data_sets/marathon_results_2017.csv")
segments = ['5K', '10K', '15K', 'Half', '25K', '30K', '35K', '40K']
results_2015['year'] = 2015
results_2016['year'] = 2016
results_2017['year'] = 2017

   - Then delete the rows which have '-' instead of a time. 

In [11]:
def clean_times(data):
    for segment in segments:
        data = data[data[segment] != '-']
    return data.copy()


results_2015 = clean_times(results_2015)
results_2016 = clean_times(results_2016)
results_2017 = clean_times(results_2017)


# Birth city elevation


- Our aim here is to obtain the elevation of the diferent cities, because the best runners are this who born in a city with a hight elevation.

- First of all we have to obtain the diferent cities on all years.

- Then use from google maps api, geolocation (to obtain the latitude and longitude of a city name) and geocoding to obtain elevation.

- Finally we will save as a csv the result, because we do not need to calculate this any time more, because it takes a lot of time and we only have only free 50k calls to the APIs and we do not want to waste its.

In [ ]:
all_cities_df = pd.concat([results_2015, results_2016, results_2017])
all_cities = all_cities_df['City'].unique()

In [ ]:
import requests
import pandas as pd
from geopy.geocoders import Nominatim


API_KEY="<HERE YOUR API KEY TO USE GOOGLE MAPS ELEVATION AND GEOCODING>"


def get_lat_lon(city):
    query = "https://maps.googleapis.com/maps/api/geocode/json?address= %s &key=%s" %((city, API_KEY))
    r = requests.get(query).json()
    lat = r['results'][0]['geometry']['location']['lat']
    lon = r['results'][0]['geometry']['location']['lng']
    return lat, lon


def get_elevation(lat, long):
    query = 'https://maps.googleapis.com/maps/api/elevation/json?locations=%s, %s&key=%s' % ((lat, long, API_KEY))
    r = requests.get(query).json()
    elevation = pd.io.json.json_normalize(r, 'results')['elevation'].values[0]
    return elevation


def process_cities_elevations(all_cities):
    todo = len(all_cities)
    city_elevation = {"cities":[], "elevations":[]}
    print(" -> [STARTING] %d will be processed." %(len(all_cities)))
    for city in all_cities:
        try:
            city = city[1]
            lat, lon = get_lat_lon(city)
            print("-> LAT : %s LON : %s obtained. " % ((lat, lon)))
            elevation = get_elevation(lat, lon)
            print("-> processed %s elevation: %s, still %d." % ((city, elevation,todo)))
        except:
            elevation = -1
        todo -= 1
        city_elevation['cities'] += [city]
        city_elevation['elevations'] += [elevation]
    pd.DataFrame(city_elevation).to_csv("city_elevations.csv" )
    
    
process_cities_elevations(all_cities)

- Now we have to add elevation to the three dataframes that we have results: 2015, 2016 and 2017.

In [12]:
def add_elevation(cities_elevations, marathon_results):
    elevations = []
    for runner in range(0, marathon_results.shape[0]):
        try:
            runner_city = marathon_results.iloc[runner]['City']
            elevation = cities_elevations[cities_elevations['cities'] == runner_city]['elevations'].iloc[0]
            elevations.append(elevation)
        except:
            elevation = -1
            elevations.append(elevation)
    marathon_results['Elevation'] = elevations
    return marathon_results[marathon_results['Elevation'] > -1.0]

In [13]:
cities_elevations = pd.read_csv("city_elevations.csv")
results_2015 = add_elevation(cities_elevations, results_2015)
results_2016 = add_elevation(cities_elevations, results_2016)
results_2017 = add_elevation(cities_elevations, results_2017)

# Add times he run the marathon 

- If he appears in all years, only two or only one.

In [15]:
runners_2015 = set(results_2015['Name'].unique())
runners_2016 = set(results_2016['Name'].unique())
runners_2017 = set(results_2017['Name'].unique())
runners_years = [runners_2015, runners_2016, runners_2017]



def times_runned(marathon_results, runners_years): 
    times_runned = []
    for runner in range(0, marathon_results.shape[0]):
        name = marathon_results.iloc[runner]['Name']
        times = 0
        for year_Results in runners_years:
            if name in year_Results:
                times += 1
        times_runned += [times]
    marathon_results['Times runned'] = times_runned
    return marathon_results


In [16]:
results_2015 = times_runned(results_2015, runners_2015)
results_2016 = times_runned(results_2016, runners_2016)
results_2017 = times_runned(results_2017, runners_2017)

# Transform gender to a not categorize attribute

- Transform M/F categorical to numerical, we will create two new features named M and F, where if M=0 and F=1 is a female or 

In [23]:
def transform_mf(marathon_results):
    M = []
    F = []
    for runner in range(0, marathon_results.shape[0]):
        MF = marathon_results.iloc[runner]['M/F']
        if MF == 'M':
            M += [1]
            F += [0]
        if MF == 'F':
            M += [0]
            F += [1]
    marathon_results['M'] = M
    marathon_results['F'] = F
    return marathon_results
        

In [24]:
results_2015 = transform_mf(results_2015)
results_2016 = transform_mf(results_2016)
results_2017 = transform_mf(results_2017)

## Climate information using open weather map

https://darksky.net


+ GET request: ```https://api.darksky.net/forecast/{API-Key}/{latitude}, {longitude}, {Unix time}```


+ And our result will be a json with a lot of information, but we are only interested on this part : 

```"currently": {
    "time": 1429520400,
    "summary": "Partly Cloudy",
    "icon": "partly-cloudy-night",
    "precipIntensity": 0,
    "precipProbability": 0,
    "temperature": 41.45,
    "apparentTemperature": 36.18,
    "dewPoint": 38.25,
    "humidity": 0.88,
    "pressure": 1022.49,
    "windSpeed": 8.34,
    "windGust": 15.65,
    "windBearing": 120,
    "cloudCover": 0.36,
    "uvIndex": 0,
    "visibility": 10
  },```
  


In [28]:
import requests
import json

API_KEY = "< HERE YOUR API KEY >"
latitude = "42.361145"
longitude = "-71.057083"

UNIX_2015 = 1429518650
UNIX_2016 = 1429345850
UNIX_2017 = 1492417850

URI_2O15 = "https://api.darksky.net/forecast/%s/%s, %s, %s" % ((API_KEY, latitude ,longitude, UNIX_2015))
URI_2016 = "https://api.darksky.net/forecast/%s/%s, %s, %s" % ((API_KEY, latitude ,longitude, UNIX_2016))
URI_2017 = "https://api.darksky.net/forecast/%s/%s, %s, %s" % ((API_KEY, latitude ,longitude, UNIX_2017))

weather_2015 = requests.get(URI_2O15).json()['currently']
weather_2016 = requests.get(URI_2016).json()['currently']
weather_2017 = requests.get(URI_2017).json()['currently']

- add weather information to the datasets.

In [39]:
def add_weather_info(marathon_results, weather_year):
    marathon_results["humidity"] = weather_year["humidity"]
    marathon_results["temperature"] = weather_year["apparentTemperature"]
    return marathon_results


results_2015 = add_weather_info(results_2015, weather_2015)
results_2016 = add_weather_info(results_2016, weather_2016)
results_2017 = add_weather_info(results_2017, weather_2017)

# Transform time to seconds

- For the linear regression it will be necessary that the time be in seconds

In [34]:
def get_sec(time_str):  
    try:
        h, m, s = time_str.split(':')
        return int(h) * 3600 + int(m) * 60 + int(s)
    except AttributeError:
        return 0
    

def transform_time(marathon_results):
    partials  = ['5K', '10K', '15K', '20K', 'Half', '25K', '30K', '35K', '40K','Official Time',]
    marathon_results = marathon_results[marathon_results[partials] != '-']
    for partial in partials:
        marathon_results[partial] = list(map(get_sec ,marathon_results[partial]))              
    return marathon_results

In [35]:
results_2015 = transform_time(results_2015)
results_2016 = transform_time(results_2016)
results_2017 = transform_time(results_2017)

In [37]:
results_2015 = pd.read_csv("2015.csv")
results_2016 = pd.read_csv("2016.csv")
results_2017 = pd.read_csv("2017.csv")

## Split diferent segments

- Each datafrem has to be splitted in eight Data Frames 5K, 10K, 15K, 20K, 25K , 30K, 35K, 40K. Also transform time to seconds and at a new parameter, current_time. This will be useful when we will want the heat and humidity at certain moment.


In [42]:
list(results_2015)

['Unnamed: 0',
 'Unnamed: 0.1',
 'Bib',
 'Name',
 'Age',
 'M/F',
 'City',
 'State',
 'Country',
 'Citizen',
 'Unnamed: 9',
 '5K',
 '10K',
 '15K',
 '20K',
 'Half',
 '25K',
 '30K',
 '35K',
 '40K',
 'Pace',
 'Proj Time',
 'Official Time',
 'Overall',
 'Gender',
 'Division',
 'year',
 'Elevation',
 'Times runned',
 'M',
 'F',
 'humidity',
 'temperature']

In [54]:
def get_sec(time_str): 
    print(time_s)
    try:
        h, m, s = time_str.split(':')
        return int(h) * 3600 + int(m) * 60 + int(s)
    except ValueError:
        return 0
        

def get_1km_partial(time, kms):
    pass


segments = ['5K', '10K', '15K', '20K', 'Half', '25K', '30K', '35K', '40K', 'Official Time']


def generate_one_segment(marathon_results, segment):
    segment_df = pd.DataFrame()
    segment_df['Times runned'] = marathon_results['Times runned']
    segment_df['Elevation'] = marathon_results['Elevation']
    segment_df['temperature'] = marathon_results['temperature']
    segment_df['M'] = marathon_results['M']
    segment_df['F'] = marathon_results['F']
    segment_df['Age'] = marathon_results['Age']
    segment_df['humidity'] = marathon_results['humidity']
    segment_df['arrival_time'] = marathon_results[segments[segment]].map(get_sec) 
    return segment_df

def segmentation(all_marathon_results):
    for segment in range(0, len(segments)):
        segment_df = pd.concat([generate_one_segment(results, segment) for results in all_marathon_results])
        segment_df.to_csv("%s.csv" % segments[segment])


In [55]:
segmentation([results_2015, results_2016, results_2017])

 - Merge all the same segments of diferent years.

In [ ]:
def merge_segments(fragment_1, fragment_2, fragment_3):
    all_segments = {}
    for segment in segments:
        all_segments[segment] = pd.concat([fragment_1[segment], fragment_2[segment], fragment_3[segment]])
    return all_segments

In [ ]:
merge_segments(segments_2015, segments_2016, segments_2017)['5K']
